In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk, filedialog
import csv
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# -------------------- Data & Logic --------------------

INITIAL_COLLEGES = [
    ("IIT Delhi", 98), ("IIT Bombay", 96), ("IIT Madras", 94), ("IIT Kanpur", 92),
    ("IIT Kharagpur", 90), ("IIT Roorkee", 88), ("IIT Guwahati", 86), ("NIT Trichy", 84),
    ("NIT Surathkal", 82), ("NIT Warangal", 80), ("NIT Calicut", 78), ("NIT Rourkela", 76),
    ("IIIT Hyderabad", 74), ("IIIT Delhi", 72), ("IIIT Bangalore", 70), ("BITS Pilani", 68),
    ("BITS Goa", 66), ("VIT Vellore", 64), ("SRM University", 62), ("Amrita University", 60),
    ("Lovely Professional University", 58), ("Manipal University", 56), ("Shiv Nadar University", 54),
    ("Jain University", 52), ("Christ University", 50), ("UPES Dehradun", 48), ("NMIMS Mumbai", 46),
    ("Ashoka University", 44), ("ISBF Delhi", 42), ("IISER Pune", 40), ("IISER Kolkata", 38),
    ("IISc Bangalore", 36), ("Jamia Millia Islamia", 34), ("Jadavpur University", 32),
    ("University of Hyderabad", 30), ("Anna University", 28), ("Osmania University", 26),
    ("Aligarh Muslim University", 24), ("BHU Varanasi", 22), ("Delhi Technological University", 20),
    ("Netaji Subhash Engineering College", 18), ("Heritage Institute of Technology", 16),
    ("IIIT Pune", 14), ("IIIT Bhopal", 12), ("IIIT Nagpur", 10), ("IIIT Lucknow", 8),
    ("IIIT Una", 6), ("IIIT Kottayam", 4), ("IIIT Ranchi", 2), ("Regional College", 0)
]

class DashboardApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("🎓 Listing Rank - College Allotment Dashboard")
        self.geometry("1000x700")
        self.config(bg="#1e1e2f")

        self.students = []
        self.colleges = INITIAL_COLLEGES.copy()
        self.cutoff_vars = {}

        style = ttk.Style(self)
        style.theme_use("clam")
        style.configure("Treeview",
                        background="#2e2e3e", fieldbackground="#2e2e3e",
                        foreground="white", rowheight=30, font=("Arial",12))
        style.configure("Treeview.Heading",
                        background="#4a4a60", foreground="white",
                        font=("Arial",13,"bold"))

        self.nb = ttk.Notebook(self)
        self.nb.pack(fill="both", expand=True, padx=10, pady=10)
        self.tab1 = ttk.Frame(self.nb)
        self.tab2 = ttk.Frame(self.nb)
        self.tab3 = ttk.Frame(self.nb)
        self.nb.add(self.tab1, text="Students & Rank")
        self.nb.add(self.tab2, text="Cut‑off Manager")
        self.nb.add(self.tab3, text="Analytics")

        self.build_tab1()
        self.build_tab2()
        self.build_tab3()

    def quick_sort(self, arr):
        if len(arr) <= 1:
            return arr
        pivot = arr[0]
        left = self.quick_sort([x for x in arr[1:] if x[1] > pivot[1]])
        right = self.quick_sort([x for x in arr[1:] if x[1] <= pivot[1]])
        return left + [pivot] + right

    def assign_college(self, marks):
        for college, cutoff in self.colleges:
            if marks >= cutoff:
                return college
        return "No College"

    def build_tab1(self):
        inp = tk.Frame(self.tab1, bg="#1e1e2f")
        inp.pack(pady=10)
        tk.Label(inp, text="Name:", font=("Helvetica",13), bg="#1e1e2f", fg="white").grid(row=0, column=0, padx=10)
        self.name_entry = tk.Entry(inp, font=("Helvetica",13), width=20, bg="#2e2e3e", fg="white")
        self.name_entry.grid(row=0, column=1, padx=10)
        tk.Label(inp, text="Marks:", font=("Helvetica",13), bg="#1e1e2f", fg="white").grid(row=0, column=2, padx=10)
        self.marks_entry = tk.Entry(inp, font=("Helvetica",13), width=20, bg="#2e2e3e", fg="white")
        self.marks_entry.grid(row=0, column=3, padx=10)
        tk.Button(inp, text="Add Student", command=self.add_student, bg="#3a7bd5", fg="white", font=("Helvetica",13), width=15).grid(row=0, column=4, padx=15)

        sr = tk.Frame(self.tab1, bg="#1e1e2f")
        sr.pack(pady=10)
        tk.Label(sr, text="Search Name/Rank:", font=("Helvetica",13), bg="#1e1e2f", fg="white").pack(side="left", padx=10)
        self.search_entry = tk.Entry(sr, font=("Helvetica",13), width=25, bg="#2e2e3e", fg="white")
        self.search_entry.pack(side="left", padx=5)
        tk.Button(sr, text="Search", command=self.search_student, bg="#00bcd4", fg="white", font=("Helvetica",13), width=12).pack(side="left", padx=5)
        tk.Button(sr, text="Show Rank List", command=self.show_rank_list, bg="#009688", fg="white", font=("Helvetica",13), width=16).pack(side="left", padx=10)

        ex = tk.Frame(self.tab1, bg="#1e1e2f")
        ex.pack(pady=10)
        tk.Button(ex, text="Import from CSV", command=self.import_csv, bg="#ff9800", fg="white", font=("Helvetica",13), width=16).pack(side="left", padx=15)
        tk.Button(ex, text="Export to CSV", command=self.export_csv, bg="#4caf50", fg="white", font=("Helvetica",13), width=16).pack(side="left", padx=15)
        tk.Button(ex, text="Clear All Data", command=self.clear_all, bg="#f44336", fg="white", font=("Helvetica",13), width=16).pack(side="left", padx=15)

        cols = ("Rank","Name","Marks","College")
        self.rank_table = ttk.Treeview(self.tab1, columns=cols, show="headings", height=15)
        for c in cols:
            self.rank_table.heading(c, text=c)
            self.rank_table.column(c, anchor="center", width=220)
        self.rank_table.pack(pady=20)

    def add_student(self):
        name = self.name_entry.get().strip()
        try:
            marks = int(self.marks_entry.get())
        except ValueError:
            return messagebox.showerror("Error","Marks must be an integer")
        self.students.append((name, marks))
        self.name_entry.delete(0, tk.END)
        self.marks_entry.delete(0, tk.END)
        messagebox.showinfo("Success", f"{name} added successfully")

    def show_rank_list(self):
        sorted_students = self.quick_sort(self.students)
        for row in self.rank_table.get_children():
            self.rank_table.delete(row)
        for i, (nm, mk) in enumerate(sorted_students, start=1):
            col = self.assign_college(mk)
            self.rank_table.insert("", "end", values=(i, nm, mk, col))

    def search_student(self):
        key = self.search_entry.get().strip().lower()
        for r in self.rank_table.get_children():
            if key in str(self.rank_table.item(r)["values"]).lower():
                self.rank_table.selection_set(r)
                self.rank_table.see(r)
                return
        messagebox.showinfo("Not Found","Student not found.")

    def export_csv(self):
        path = filedialog.asksaveasfilename(defaultextension=".csv")
        if not path: return
        with open(path, "w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["Rank","Name","Marks","College"])
            for r in self.rank_table.get_children():
                w.writerow(self.rank_table.item(r)["values"])
        messagebox.showinfo("Exported","Rank list exported successfully!")

    def clear_all(self):
        if messagebox.askyesno("Clear All","Are you sure?"):
            self.students.clear()
            for r in self.rank_table.get_children():
                self.rank_table.delete(r)
            messagebox.showinfo("Cleared","All data cleared!")

    def import_csv(self):
        path = filedialog.askopenfilename(filetypes=[("CSV","*.csv")])
        if not path: return
        try:
            with open(path, newline="") as f:
                reader = csv.reader(f)
                hdr = next(reader, None)
                ni, mi = (hdr.index("Name"), hdr.index("Marks")) if hdr and "Name" in hdr and "Marks" in hdr else (0, 1)
                self.students.clear()
                count = 0
                for row in reader:
                    if len(row) > mi:
                        try:
                            m = int(row[mi])
                        except ValueError:
                            continue
                        self.students.append((row[ni].strip(), m))
                        count += 1
            self.show_rank_list()
            messagebox.showinfo("Imported", f"{count} students imported successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load CSV:\n{e}")

    def build_tab2(self):
        canvas = tk.Canvas(self.tab2, bg="#1e1e2f")
        sb = ttk.Scrollbar(self.tab2, orient="vertical", command=canvas.yview)
        frame = ttk.Frame(canvas)
        frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0,0), window=frame, anchor="nw")
        canvas.configure(yscrollcommand=sb.set)
        canvas.pack(side="left", fill="both", expand=True)
        sb.pack(side="right", fill="y")

        for cname, cut in self.colleges:
            row = ttk.Frame(frame, padding=5)
            row.pack(fill="x", pady=2)
            ttk.Label(row, text=cname, width=30).pack(side="left")
            var = tk.IntVar(value=cut)
            self.cutoff_vars[cname] = var
            ttk.Entry(row, textvariable=var, width=5).pack(side="left", padx=10)

        ttk.Button(frame, text="Apply Cut‑offs", command=self.apply_cutoffs).pack(pady=15)

    def apply_cutoffs(self):
        self.colleges = [(c, self.cutoff_vars[c].get()) for c,_ in self.colleges]
        self.show_rank_list()
        messagebox.showinfo("Updated","Cut‑offs applied.")

    def build_tab3(self):
        ttk.Button(self.tab3, text="Refresh Chart", command=self.draw_histogram).pack(pady=5)
        self.fig = Figure(figsize=(6,4))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.tab3)
        self.canvas.get_tk_widget().pack(fill="both", expand=True)

    def draw_histogram(self):
        self.ax.clear()
        marks = [m for _, m in self.students]
        if marks:
            bins = range(min(marks), max(marks)+5, 5)
            n, bins, patches = self.ax.hist(marks, bins=bins, color="skyblue", edgecolor="black")
            avg = sum(marks) / len(marks)
            self.ax.axvline(avg, color='red', linestyle='--', linewidth=2, label=f'Average: {avg:.2f}')
            self.ax.set_title("📊 Marks Distribution", fontsize=14, fontweight='bold', color="black")
            self.ax.set_xlabel("Marks", fontsize=12, color="black")
            self.ax.set_ylabel("Number of Students", fontsize=12, color="black")
            self.ax.legend()

            for rect in patches:
                height = rect.get_height()
                if height > 0:
                    self.ax.annotate(f"{int(height)}", xy=(rect.get_x() + rect.get_width()/2, height),
                                     xytext=(0, 5), textcoords="offset points",
                                     ha="center", fontsize=9, color="black")

            self.ax.set_facecolor("white")
            self.fig.patch.set_facecolor("white")
            self.ax.tick_params(colors='black')
            self.ax.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
        self.canvas.draw()

if __name__ == "__main__":
    app = DashboardApp()
    app.mainloop()
    